In [1]:
import numpy as np
import pandas as pd

import swisslandstats as sls

In [2]:
vaud_ldf = sls.read_csv('data/vaud_ldf.csv')

Reading multiindexed csv with blanks (see https://stackoverflow.com/questions/30322581/pandas-read-multiindexed-csv-with-blanks)

In [3]:
# TODO: fix ugly hardcoded names

def correct_columns_from_csv(df):
    columns = pd.DataFrame(df.columns.tolist())
    columns.loc[columns[0].str.startswith('Unnamed'), 0] = np.nan
    columns[0] = columns[0].fillna(method='ffill')
    mask = pd.isnull(columns[0])
    columns[0] = columns[0].fillna('')
    columns.loc[mask, [0,1]] = columns.loc[mask, [1,0]].values
    df_idx = pd.MultiIndex.from_tuples(columns.to_records(index=False).tolist(), names=['stock', 'year'])
    df_idx = df_idx.set_levels(df_idx.levels[1].astype(int), level='year')
    return df_idx

def correct_index_from_csv(df):
    index = pd.DataFrame(df.index.tolist())
    index[0] = index[0].fillna(method='ffill')
    df.index = pd.MultiIndex.from_tuples(index.to_records(index=False).tolist(), names=['LULC', 'prodreg', 'elevation'])

    # index lulc as int
    df.index = df.index.set_levels(df.index.levels[0].astype(int), level='LULC')
    
    # if map_prodreg:
    #     # index prodreg names instead of numbers
    #     prodreg_name_map = {1: 'Jura', 2: 'Central Plateau', 3: 'Pre Alps', 4: 'Alps', 5: 'Southern Alps'}
    #     df.index = df.index.set_levels(df.index.levels[1].map(lambda x: prodreg_name_map[x]), level='prodreg')
    
    return df.index

In [4]:
variable_coeff_df = pd.read_csv('data/variable_coeffs.csv', header=[0,1], index_col=[0,1,2])

variable_coeff_df.columns = correct_columns_from_csv(variable_coeff_df)
variable_coeff_df.index = correct_index_from_csv(variable_coeff_df)

variable_coeff_df = variable_coeff_df.unstack(0).stack(1)

In [5]:
constant_coeff_df = pd.read_csv('data/constant_coeffs.csv', header=0, index_col=[0,1,2])

constant_coeff_df.index = correct_index_from_csv(constant_coeff_df)
constant_coeff_df = constant_coeff_df.unstack(0)

## Get yearly coefficient tables

In [6]:
# TODO: DRY the stratification strategy with Python iterators

def get_stratum_ser(ldf, lulc_column, year_column):
    stratum_ser = pd.Series(0, index=ldf.index, name='stratum')
    for (prodreg, elev, orgsoil, lulc, year), stratum_df in ldf.groupby(
            ['prodreg', 'elevation', 'orgsoil', lulc_column, year_column]):
        stratum_code = int(str(prodreg) + str(elev) + str(orgsoil) + str(lulc))
        stratum_ser.loc[stratum_df.index] = stratum_code

    return stratum_ser
    
def get_coeff_df(ldf, lulc_column, year_column, variable_coeff_df, constant_coeff_df):
    
    variable_lulc_columns = list(variable_coeff_df.columns.levels[1])
    constant_lulc_columns = list(constant_coeff_df.columns.levels[1])

    indep_stock_columns = list(variable_coeff_df.columns.levels[0][:3])    

    coeff_df = pd.DataFrame(columns=indep_stock_columns + ['C_soil'])
    
    for (prodreg, elev, orgsoil, lulc, year), stratum_df in ldf.groupby(
            ['prodreg', 'elevation', 'orgsoil', lulc_column, year_column]):
        stratum_lulc_code = int(str(prodreg) + str(elev) + str(orgsoil) + str(lulc))
        # ldf.loc[stratum_df.index, stratum_lulc_col] = stratum_lulc_code
        if lulc in variable_lulc_columns:
            coeff_row = variable_coeff_df.loc[(prodreg, elev, year)]
        else:
            coeff_row = constant_coeff_df.loc[(prodreg, elev)]
        for stock_col in indep_stock_columns:
            coeff_df.loc[stratum_lulc_code, stock_col] = coeff_row[(stock_col, lulc)]
        if orgsoil == 0:
            coeff_df.loc[stratum_lulc_code, 'C_soil'] = coeff_row[('C_mineral_soil', lulc)]
        else:
            coeff_df.loc[stratum_lulc_code, 'C_soil'] = coeff_row[('C_organic_soil', lulc)]
    
    return coeff_df

## Enter Invest

In [7]:
from natcap.invest import carbon

In [8]:
import os

workspace_dir = 'invest_carbon_output'
if not os.path.exists(workspace_dir):
    os.makedirs(workspace_dir)

In [9]:
import rasterio

In [10]:
# Map to Invest names

stock_map = {'C_dead_wood': 'C_dead', 'C_litter': 'C_below', 'C_living_biomass': 'C_above', 'C_soil': 'C_soil'}

In [19]:
# ACHTUNG: ensure that the column order is consistent for lulc and year columns
lulc_columns = vaud_ldf.columns[vaud_ldf.columns.str.startswith('LULC')]
year_columns = vaud_ldf.columns[vaud_ldf.columns.str.startswith('FJ')]

for lulc_column, year_column in zip(lulc_columns, year_columns):
    # Map the strata to their corresponding pixels
    stratum_lulc_column = 'STRAT_' + lulc_column
    vaud_ldf[stratum_lulc_column] = get_stratum_ser(vaud_ldf, lulc_column, year_column)

    # Export GeoTiff of the LULC strata
    lulc_cur_path = os.path.join(workspace_dir, lulc_column + '_input.tif')
    vaud_ldf.to_geotiff(lulc_cur_path, stratum_lulc_column, dtype=rasterio.uint32)

    # Get stratified coefficients (with columns mapped to invest names)
    coeff_df = get_coeff_df(vaud_ldf, lulc_column, year_column, variable_coeff_df, constant_coeff_df)
    coeff_df.columns = coeff_df.columns.map(lambda col: stock_map[col])
    # lulc code column must be named lucode (for the invest carbon model)
    coeff_df.index = coeff_df.index.set_names('lucode')
    # export it to csv
    carbon_pools_path = os.path.join(workspace_dir, 'coeffs_' + lulc_column)
    coeff_df.to_csv(carbon_pools_path)


    args = {'workspace_dir': workspace_dir, 'results_suffix': lulc_column, 'lulc_cur_path': lulc_cur_path, 'carbon_pools_path': carbon_pools_path}
    carbon.execute(args)

## Postprocessing

In [13]:
import glob

In [27]:
# fig, axes = plt.subplots(len(lulc_columns), 1, figsize=(6, 6* len(lulc_columns)))
for lulc_column in lulc_columns:
    invest_output_fp = glob.glob(os.path.join(workspace_dir, '*' + lulc_column + '.tif'))[0]
    with rasterio.open(invest_output_fp, 'r') as src:
        arr = src.read(1)
    # ax.imshow(arr)
    print(np.nansum(arr))

32715770.0
32942260.0
33674450.0
34080136.0
